<a href="https://colab.research.google.com/github/graviraja/100-Days-of-NLP/blob/applications%2Fquestion-answering/applications/question-answering/Question%20Answering%20using%20DMN%20Plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### bAbI Dataset

In [1]:
!wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz

--2020-07-31 18:19:37--  http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz
Resolving www.thespermwhale.com (www.thespermwhale.com)... 69.65.3.213
Connecting to www.thespermwhale.com (www.thespermwhale.com)|69.65.3.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15719851 (15M) [application/x-gzip]
Saving to: ‘tasks_1-20_v1-2.tar.gz’

tasks_1-20_v1-2.tar 100%[===================>]  14.99M  6.40MB/s    in 2.3s    

2020-07-31 18:19:40 (6.40 MB/s) - ‘tasks_1-20_v1-2.tar.gz’ saved [15719851/15719851]



In [2]:
!tar -xzf tasks_1-20_v1-2.tar.gz

In [3]:
!ls tasks_1-20_v1-2/en-10k/

qa10_indefinite-knowledge_test.txt   qa1_single-supporting-fact_test.txt
qa10_indefinite-knowledge_train.txt  qa1_single-supporting-fact_train.txt
qa11_basic-coreference_test.txt      qa20_agents-motivations_test.txt
qa11_basic-coreference_train.txt     qa20_agents-motivations_train.txt
qa12_conjunction_test.txt	     qa2_two-supporting-facts_test.txt
qa12_conjunction_train.txt	     qa2_two-supporting-facts_train.txt
qa13_compound-coreference_test.txt   qa3_three-supporting-facts_test.txt
qa13_compound-coreference_train.txt  qa3_three-supporting-facts_train.txt
qa14_time-reasoning_test.txt	     qa4_two-arg-relations_test.txt
qa14_time-reasoning_train.txt	     qa4_two-arg-relations_train.txt
qa15_basic-deduction_test.txt	     qa5_three-arg-relations_test.txt
qa15_basic-deduction_train.txt	     qa5_three-arg-relations_train.txt
qa16_basic-induction_test.txt	     qa6_yes-no-questions_test.txt
qa16_basic-induction_train.txt	     qa6_yes-no-questions_train.txt
qa17_positional-reasoning_test.

In [4]:
train_file = 'tasks_1-20_v1-2/en-10k/qa5_three-arg-relations_train.txt'
test_file = 'tasks_1-20_v1-2/en-10k/qa5_three-arg-relations_test.txt'

### Imports

In [48]:
import re
import os
import time
import nltk
import random
import pickle
import numpy as np

from collections import Counter
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable

from sklearn.model_selection import train_test_split


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
with open(train_file) as f:
    train_file_data = f.readlines()

In [8]:
train_file_data = [d[:-1] for d in train_file_data]

In [9]:
train_file_data[:30]

['1 Bill travelled to the office.',
 '2 Bill picked up the football there.',
 '3 Bill went to the bedroom.',
 '4 Bill gave the football to Fred.',
 '5 What did Bill give to Fred? \tfootball\t4',
 '6 Fred handed the football to Bill.',
 '7 Jeff went back to the office.',
 '8 Who received the football? \tBill\t6',
 '9 Bill travelled to the office.',
 '10 Bill got the milk there.',
 '11 Who received the football? \tBill\t6',
 '12 Fred travelled to the garden.',
 '13 Fred went to the hallway.',
 '14 Bill journeyed to the bedroom.',
 '15 Jeff moved to the hallway.',
 '16 Jeff journeyed to the bathroom.',
 '17 Bill journeyed to the office.',
 '18 Fred travelled to the bathroom.',
 '19 Mary journeyed to the kitchen.',
 '20 Jeff took the apple there.',
 '21 Jeff gave the apple to Fred.',
 '22 Who did Jeff give the apple to? \tFred\t21',
 '23 Bill went back to the bathroom.',
 '24 Bill left the milk.',
 '25 Who received the apple? \tFred\t21',
 '1 Mary travelled to the garden.',
 '2 Mary journe

In [10]:
def load_data(data):
    qa_data = []
    fact = []
    qa = []

    for d in data:
        index = d.split()[0]
        if index == "1":
            fact = []
            qa = []
        if '?' in d:
            temp = d.split('\t')
            q = temp[0].strip().replace('?', '').split()[1:] + ['?']
            a = temp[1].split()
            stemp = deepcopy(fact)
            qa_data.append([stemp, q, a])
        else:
            tokens = d.replace('.', '').split()[1:] + ['</s>']
            fact.append(tokens)
    return qa_data

In [11]:
train_data = load_data(train_file_data)

In [12]:
len(train_data)

10000

In [13]:
train_data[0], train_data[1]

([[['Bill', 'travelled', 'to', 'the', 'office', '</s>'],
   ['Bill', 'picked', 'up', 'the', 'football', 'there', '</s>'],
   ['Bill', 'went', 'to', 'the', 'bedroom', '</s>'],
   ['Bill', 'gave', 'the', 'football', 'to', 'Fred', '</s>']],
  ['What', 'did', 'Bill', 'give', 'to', 'Fred', '?'],
  ['football']],
 [[['Bill', 'travelled', 'to', 'the', 'office', '</s>'],
   ['Bill', 'picked', 'up', 'the', 'football', 'there', '</s>'],
   ['Bill', 'went', 'to', 'the', 'bedroom', '</s>'],
   ['Bill', 'gave', 'the', 'football', 'to', 'Fred', '</s>'],
   ['Fred', 'handed', 'the', 'football', 'to', 'Bill', '</s>'],
   ['Jeff', 'went', 'back', 'to', 'the', 'office', '</s>']],
  ['Who', 'received', 'the', 'football', '?'],
  ['Bill']])

In [14]:
train_data, eval_data = train_test_split(train_data, test_size=0.2, random_state=42)
len(train_data), len(eval_data)

(8000, 2000)

### Vocabulary

In [15]:
PAD_TOKEN = 0
START_TOKEN = 1
END_TOKEN = 2
UNK_TOKEN = 3

In [16]:
class Vocab(object):
    def __init__(self):
        self.word2id = {
            "<s>": START_TOKEN,
            "</s>": END_TOKEN,
            "<pad>": PAD_TOKEN,
            "<unk>": UNK_TOKEN
        }

        self.id2word = dict((id, word) for word, id in self.word2id.items())
        self.length = 4
    
    def add_word(self, word):
        if word not in self.word2id:
            self.word2id[word] = self.length
            self.id2word[self.length] = word
            self.length += 1
    
    def __len__(self):
        return self.length
    
    def __call__(self, word):
        if not word in self.word2id:
            return self.word2id['<unk>']
        return self.word2id[word]

In [17]:
def build_vocab(data, min_count=5):
    facts, questions, answers = list(zip(*data))
    vocab = Vocab()
    counter = Counter()

    for fact in facts:
        for f in fact:
            counter.update(f)
    
    for q in questions:
        counter.update(q)
    
    for a in answers:
        counter.update(a)
    
    for word, count in counter.most_common():
        if count >= min_count:
            vocab.add_word(word)
    
    vocab_file = 'vocab.pkl'
    print("saving vocabulary...")
    with open(vocab_file, 'wb') as f:
        pickle.dump(vocab, f)

In [18]:
def load_vocab():
    with open('vocab.pkl', 'rb') as f:
        vocab = pickle.load(f)
    return vocab

In [19]:
build_vocab(train_data)

saving vocabulary...


In [20]:
vocab = load_vocab()
len(vocab)

44

In [21]:
list(vocab.word2id.keys())

['<s>',
 '</s>',
 '<pad>',
 '<unk>',
 'the',
 'to',
 'Mary',
 'Fred',
 'Bill',
 'Jeff',
 'went',
 'there',
 'apple',
 'football',
 'milk',
 'moved',
 'travelled',
 'journeyed',
 'back',
 'garden',
 'office',
 'hallway',
 'kitchen',
 'bathroom',
 'bedroom',
 'gave',
 '?',
 'took',
 'passed',
 'grabbed',
 'got',
 'picked',
 'up',
 'handed',
 'Who',
 'left',
 'dropped',
 'put',
 'down',
 'discarded',
 'did',
 'give',
 'What',
 'received']

### Dataset Creation

In [22]:
def get_batch(data, batch_size=64):
    start_idx = 0
    end_idx = batch_size
    while end_idx < len(data):
        batch = data[start_idx: end_idx]
        start_idx = end_idx
        end_idx = end_idx + batch_size
        yield batch
    
    if end_idx >= len(data):
        batch = data[start_idx:]
        yield batch

In [23]:
def prepare_seq(seq):
    idxs = [vocab(tok) for tok in seq]
    return torch.LongTensor(idxs)

In [24]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [25]:
def pad_batch(batch):
    fact, ques, ans = list(zip(*batch))

    max_facts = max([len(f) for f in fact])
    max_fact_len = max([f.size(1) for f in flatten(fact)])
    max_ques_len = max([q.size(1) for q in ques])
    max_ans_len = max([a.size(1) for a in ans])

    facts, questions, answers = [], [], []

    for i in range(len(batch)):
        fact_t = []
        fact_l = []
        for j in range(len(fact[i])):
            if fact[i][j].size(1) < max_ques_len:
                fact_l.append(fact[i][j].size(1))
                fact_t.append(torch.cat([fact[i][j], Variable(torch.LongTensor([[vocab('<pad>')] * (max_fact_len - fact[i][j].size(1))])).view(1, -1)], 1))
            else:
                fact_t.append(fact[i][j])
                fact_l.append(fact[i][j].size(1))
        
        while len(fact_t) < max_facts:
            fact_t.append(Variable(torch.LongTensor([vocab('<pad>')] * max_fact_len)).view(1, -1))

        fact_t = torch.cat(fact_t)
        facts.append(fact_t.unsqueeze(0))

        # padding question
        if ques[i].size(1) < max_ques_len:
            questions.append(torch.cat([ques[i], Variable(torch.LongTensor([[vocab('<pad>')] * (max_ques_len - ques[i].size(1))])).view(1, -1)], 1))
        else:
            questions.append(ques[i])
        
        # padding answer
        if ans[i].size(1) < max_ans_len:
            answers.append(torch.cat([ans[i], Variable(torch.LongTensor([[vocab('<pad>')] * (max_ans_len - ans[i].size(1))])).view(1, -1)], 1))
        else:
            answers.append(ans[i])

    facts = torch.cat(facts).to(device)
    questions = torch.cat(questions).to(device)
    answers = torch.cat(answers).to(device)
    
    return facts, questions, answers

In [26]:
for t in train_data:
    for i, fact in enumerate(t[0]):
        t[0][i] = prepare_seq(fact).view(1, -1)
    t[1] = prepare_seq(t[1]).view(1, -1)
    t[2] = prepare_seq(t[2]).view(1, -1)

In [27]:
for t in eval_data:
    for i, fact in enumerate(t[0]):
        t[0][i] = prepare_seq(fact).view(1, -1)
    t[1] = prepare_seq(t[1]).view(1, -1)
    t[2] = prepare_seq(t[2]).view(1, -1)

### Question Module

In [28]:
class QuestionModule(nn.Module):
    def __init__(self, hidden_size, embedding, dropout=0.3):
        super().__init__()

        self.embedding = embedding
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, questions):
        # questions => [batch_size, max_ques_len]
        
        embed = self.embedding(questions)
        embed = self.dropout(embed)
        # embed => [batch_size, max_ques_len, hid_dim]

        output, hidden = self.gru(embed)
        # output => [batch_size, max_ques_len, hid_dim]
        # hidden => [1, batch_size, hid_dim]

        question_rep = hidden.transpose(0, 1)
        # question_rep => [batch_size, 1, hid_dim]

        return question_rep

### Input Module

In [66]:
class InputModule(nn.Module):
    def __init__(self, hidden_size, embedding, dropout=0.3):
        super().__init__()

        self.hidden_size = hidden_size
        self.embedding = embedding
        # input fusion layer for exchaning information between sentences
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True, batch_first=True)

        self.dropout = nn.Dropout(dropout)
    
    def positional_encoder(self, embedded):
        # embedded => [batch_size, num_sentences, num_tokens, hid_dim]

        batch_size, num_sentences, num_tokens, emb_length = embedded.size()
        l = []
        M = num_tokens - 1
        D = emb_length
        for j in range(num_tokens):
            x = []
            for d in range(emb_length):
                x.append((1 - j / M) - (d/D)*(1 - 2*j/M))
            l.append(x)
        
        l = Variable(torch.FloatTensor(l)).to(device)
        # l => [num_tokens, emb_length]
        l = l.unsqueeze(0)
        l = l.unsqueeze(1)
        # l => [1, 1, num_tokens, emb_length]

        mat = embedded * l
        # mat => [batch_size, num_sentences, num_tokens, hid_dim]
        encoded_facts = torch.sum(mat, dim=2).squeeze(2)
        # encoded_facts => [batch_size, num_sentences, hid_dim]

        return encoded_facts

    def forward(self, facts):
        # facts => [batch_size, num_sentences, num_tokens]

        batch_size, num_sentences, num_tokens = facts.size()

        facts = facts.view(batch_size, -1)
        embed = self.embedding(facts)
        embed = embed.view(batch_size, num_sentences, num_tokens, -1)
        # embed => [batch_size, num_sentences, num_tokens, hid_dim]

        positional_encoded = self.positional_encoder(embed)
        positional_encoded = self.dropout(positional_encoded)
        # positional_encoded => [batch_size, num_sentences, hid_dim]

        outputs, hidden = self.gru(positional_encoded)
        # outputs => [batch_size, num_sentences, hid_dim * 2]
        # hidden => [2, batch_size, hid_dim]

        # combine the forward and backward gru outputs of each sentence
        combined = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        # combined => [batch_size, num_sentences, hid_dim]

        return combined

### Episodic Memory Module

#### Attention Cell

In [81]:
class AttentionGRUCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.Wr = nn.Linear(input_size, hidden_size)
        self.Ur = nn.Linear(hidden_size, hidden_size)
        self.W = nn.Linear(input_size, hidden_size)
        self.U = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, fact, hidden, gate):
        # fact => [batch_size, hid_dim]
        # hidden => [batch_size, hid_dim]
        # gate => [batch_size]

        r = F.sigmoid(self.Wr(fact) + self.Ur(hidden))
        h_tilda = F.tanh(self.W(fact) + r * self.U(hidden))
        
        gate = gate.unsqueeze(1).expand_as(h_tilda)
        # gate => [batch_size, hid_dim]
        
        hidden = gate * h_tilda + (1 - gate) * hidden
        return hidden 

#### Attention GRU

In [82]:
class AttentionGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.attn_gru = AttentionGRUCell(input_size, hidden_size)
    
    def forward(self, facts, gates):
        # facts => [batch_size, num_sentences, hid_dim]
        # gates => [batch_size, num_sentences]

        num_sentences = facts.size(1)
        init_hidden = Variable(torch.zeros(self.hidden_size)).to(device)
        # init_hidden => [hid_dim]

        for sent in range(num_sentences):
            fact = facts[:, sent, :]
            gate = gates[:, sent]

            if sent == 0:
                hidden = init_hidden.unsqueeze(0).expand_as(fact)
                # hidden => [batch_size, hid_dim]
            hidden = self.attn_gru(fact, hidden, gate)
        context = hidden
        return context

#### Memory Module

In [83]:
class MemoryModule(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.gru = AttentionGRU(hidden_size, hidden_size)
        self.W1 = nn.Linear(4 * hidden_size, hidden_size)
        self.W2 = nn.Linear(hidden_size, 1)
        self.W_mem = nn.Linear(3 * hidden_size, hidden_size)
    
    def gate_matrix(self, facts, questions, prev_mem):
        # facts => [batch_size, num_sentences, hid_dim]
        # questions => [batch_size, 1, hid_dim]
        # prev_mem => [batch_size, 1, hid_dim]

        batch_size = facts.size(0)
        questions = questions.expand_as(facts)
        prev_mem = prev_mem.expand_as(facts)

        z = torch.cat(
            [
             facts * questions,
             facts * prev_mem,
             torch.abs(facts - questions),
             torch.abs(facts - prev_mem)
            ],
            dim=2
        )
        # z => [batch_size, num_sentences, 4 * hid_dim]
        z = z.view(-1, 4 * self.hidden_size)
        z = self.W2(F.tanh(self.W1(z)))
        z = z.view(batch_size, -1)
        gates = F.softmax(z)

        return gates

    def forward(self, facts, questions, prev_mem):
        # facts => [batch_size, num_sentences, hid_dim]
        # questions => [batch_size, 1, hid_dim]
        # prev_mem => [batch_size, 1, hid_dim]
        gates = self.gate_matrix(facts, questions, prev_mem)
        context = self.gru(facts, gates)

        concat = torch.cat([prev_mem.squeeze(1), context, questions.squeeze(1)], dim=1)
        next_memory = F.relu(self.W_mem(concat))
        next_memory = next_memory.unsqueeze(1)

        return next_memory

### Answer Module

In [84]:
class AnswerModule(nn.Module):
    def __init__(self, vocab_size, hidden_size, dropout=0.3):
        super().__init__()

        self.W = nn.Linear(2 * hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, memory, questions):
        # memory => [batch_size, 1, hid_dim]
        # questions => [batch_size, 1, hid_dim]
        combined = torch.cat([memory, questions], dim=2).squeeze(1)
        out = self.W(combined)
        # out => [batch_size, vocab_size]

        return out

### DMN+

In [85]:
class DMNPlus(nn.Module):
    def __init__(self, hidden_size, vocab_size, num_episodes=3):
        super().__init__()

        self.num_episodes = num_episodes
        self.word_embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=0)

        self.input_module = InputModule(hidden_size, self.word_embedding)
        self.question_module = QuestionModule(hidden_size, self.word_embedding)
        self.episodic_memory_module = MemoryModule(hidden_size)
        self.answer_module = AnswerModule(vocab_size, hidden_size)

    def forward(self, context, questions):
        # context => [batch_size, num_sentences, num_tokens]
        # questions => [batch_size, num_tokens]c

        encoded_facts = self.input_module(context)
        encoded_ques = self.question_module(questions)

        memory = encoded_ques
        for episode in range(self.num_episodes):
            memory = self.episodic_memory_module(encoded_facts, encoded_ques, memory)
        
        preds = self.answer_module(memory, encoded_ques)
        return preds

In [86]:
HIDDEN_SIZE = 80
NUM_EPISODES = 3

model = DMNPlus(HIDDEN_SIZE, len(vocab), NUM_EPISODES).to(device)

In [87]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [88]:
def train(model, dataset):
    model.train()

    epoch_loss = 0
    total = 0
    for i, batch in enumerate(get_batch(dataset)):
        facts, questions, answers = pad_batch(batch)

        model.zero_grad()
        predictions = model(facts, questions)
        loss = criterion(predictions, answers.view(-1))
        
        epoch_loss += loss.item()
        total += 1

        loss.backward()
        optimizer.step()
    
    return epoch_loss / total


In [89]:
def evaluate(model, dataset):
    model.eval()

    epoch_loss = 0
    total = 0
    with torch.no_grad():
        for i, batch in enumerate(get_batch(dataset)):
            facts, questions, answers = pad_batch(batch)

            predictions = model(facts, questions)
            loss = criterion(predictions, answers.view(-1))
            
            epoch_loss += loss.item()
            total += 1
    
    return epoch_loss / total

In [90]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [91]:
n_epochs = 10

best_loss = float('inf')
no_improvement = 0
patience = 3

for epoch in range(n_epochs):
    start_time = time.time()
    train_loss = train(model, train_data)
    eval_loss = evaluate(model, eval_data)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {eval_loss:.3f}')

    if eval_loss < best_loss:
        best_loss = eval_loss
        no_improvement = 0
        torch.save(model.state_dict(), 'model.pt')
    else:
        no_improvement += 1
    
    if no_improvement == patience:
        break

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01 | Time: 0m 53s
	Train Loss: 1.644 | Val. Loss: 1.289
Epoch: 02 | Time: 0m 52s
	Train Loss: 1.258 | Val. Loss: 1.238
Epoch: 03 | Time: 0m 51s
	Train Loss: 1.065 | Val. Loss: 0.748
Epoch: 04 | Time: 0m 51s
	Train Loss: 0.535 | Val. Loss: 0.497
Epoch: 05 | Time: 0m 52s
	Train Loss: 0.375 | Val. Loss: 0.349
Epoch: 06 | Time: 0m 51s
	Train Loss: 0.308 | Val. Loss: 0.334
Epoch: 07 | Time: 0m 53s
	Train Loss: 0.295 | Val. Loss: 0.324
Epoch: 08 | Time: 0m 51s
	Train Loss: 0.286 | Val. Loss: 0.312
Epoch: 09 | Time: 0m 52s
	Train Loss: 0.270 | Val. Loss: 0.320
Epoch: 10 | Time: 0m 51s
	Train Loss: 0.268 | Val. Loss: 0.316
